本来是打算用数据清洗+数据增强来做  
鉴于现在的数据入库只入库init_doctree，并没有数据增强。es部分的文档入库也需要再重构一下。同时agent重构的pr还未合入。此次改动只做数据清洗。


看看html的清洗效果    
[清洗前的网页](./original_html.html)    
[清洗后的网页](./simplified_html.html)


核心逻辑：用[simplify_html](../../kernel/utils/doctree.py#L348)清洗html标签，并且增强数据

一个简单实验来显示当前的效果  
    针对[该法案](https://eur-lex.europa.eu/legal-content/EN/TXT/HTML/?uri=CELEX%3A32023R0956)，设计了4个均需要参考附录的问题
    

| 问题 | 参考 | 说明 |
| -- | -- | -- |
| Q1：本条例不适用于原产于哪些国家的货物？ |     annex III  |  直接用annex原文即可回答，数据加强与否应该没区别 |
| Q2：附录中提到的释放一氧化二氮气体的货物有哪些？ |     annex I  |  直接用annex原文即可回答，数据加强与否应该没区别 |
| Q3：simple goods是什么？ |     annex IV  |  直接用annex原文即可回答，数据加强与否应该没区别 |
| Q4：会排放perfluorocarbons（全氟化碳）的货物有哪些？要给出其CN编码 |     annex II  |  直接用annex原文即可回答，数据加强与否应该没区别 |


实验结果记录   
本条例不适用于原产于哪些国家的货物？  
根据《条例 32023R0956》的附录 III，本条例不适用于以下国家原产的货物：冰岛、列支敦士登、挪威和瑞士。此外，这一条例也不适用于以下地区的货物：比辛根、赫利戈兰、利维尼奥、休达和梅利利亚（来源：Regulation 32023R0956, ANNEX III）。   



附录中提到的释放一氧化二氮气体的货物有哪些？   
在附录中，没有提到释放一氧化二氮气体的具体货物。根据提供的上下文信息，附录主要涉及水泥和铁钢中与二氧化碳相关的货物和温室气体（Article 1, ANNEX I）。如果需要更具体的信息，可能需要查阅其他相关文件或法规。


simple goods是什么？  
Simple goods are defined as goods produced in a production process that exclusively requires input materials (precursors) and fuels with zero embedded emissions (Regulation 32023R0956, Annex IV).


会排放perfluorocarbons（全氟化碳）的货物有哪些？要给出其CN编码   
The goods that emit perfluorocarbons are primarily related to aluminum products. Their CN codes include 7601 for unwrought aluminum, 7603 for aluminum powders and flakes, 7604 for aluminum bars, rods, and profiles, 7605 for aluminum wire, and 7606 for aluminum plates, sheets, and strips, of a thickness exceeding 0.2 mm (Regulation 32023R0956, Annex I and II).   
**只给出了部分答案，原因是召回的文档只有一半（入库时被截断了）。由此引入了es数据入库的代码重构工作。**

----------------------------------分割线----------------------------------  
后面是复现实验的代码，可以选择性运行。赶时间可以不看了


In [3]:
# 环境准备
import sys
from pathlib import Path
project_directory = Path('/home/jaxxs/LawAgent')  # 替换成你的实际项目路径
sys.path.append(str(project_directory))

In [5]:
index_name = "test_lawagent_regulations_zy"
es_index = 'test.lawagent.regulations_zy'

from kernel.connector.es.es_client import ElasticsearchClient
from kernel.utils.env_manager import initialize_environment
from pathlib import Path
from kernel.utils.memory_runner import DATABASE, DocNode
from kernel.utils.doctree import check_init_doctree
import json
# project_directory = Path(__file__).resolve().parents[2]
initialize_environment(project_directory)
from kernel.connector.es.es_utils import *
class MyElasticsearchClient(ElasticsearchClient):
    def __init__(self, index="regulation"):
        super().__init__(index)
        self.index_name = es_index
        self._index = es_index
    def insert_single_regulation(self, celex_number: str, regulation_name: str, database: DATABASE):
        """利用初始doctree，将单篇法案入库"""
        
        init_doctree_path = check_init_doctree(celex_number)
        with init_doctree_path.open("r", encoding="utf-8") as f:
            serialized_root_node = json.load(f)
        es_root_node = DocNode.deserialize(serialized_root_node, database)
        # 指定入库时的index（建议放到config里，和es的参数放一起）
        index_name = es_index
        self.create_index_and_mapping(index_name)
        logger = logging.getLogger(__name__)
        logger.info("Insert regulation %s to es.", celex_number)
        doc_id = 0
        # 背景信息入库
        pre_part = es_root_node.children[0]
        doc_id = pre_part_to_es(pre_part, self, index_name, celex_number, regulation_name, doc_id)
        # 正文信息入库
        main_part = es_root_node.children[1]
        doc_id = main_part_to_es(main_part, self, index_name, celex_number, regulation_name, doc_id)
        # post部分信息入库
        post_part = es_root_node.children[2]
        doc_id = post_part_to_es(post_part, self, index_name, celex_number, regulation_name, doc_id)
        # 附录部分信息入库
        annex_part = es_root_node.children[3]
        _ = annex_part_to_es(annex_part, self, index_name, celex_number, regulation_name, doc_id)

In [6]:
tmpes = MyElasticsearchClient()
# tmpes.create_index_and_mapping()
#查看该index是否存在
indices = tmpes.es.cat.indices(v=True)
print(indices)
# 然后插入法案
# tmpes.insert_single_regulation("32023R0956", "Regulation 32023R0956", DATABASE()) 


2024-12-12 16:41:44,278 - elastic_transport.transport - INFO - GET https://elastic.kclab.cloud:9200/_cat/indices?v=true [status:200 duration:0.112s]


health status index                                                     uuid                   pri rep docs.count docs.deleted store.size pri.store.size dataset.size
green  open   test.finance.database.wind.ashareannfinancialindicator    PEunkn1oTc-4jSVs0y2c5Q   1   1     309856            0    373.9mb        186.9mb      186.9mb
green  open   test.finance.database.wind.asharefairvaluechange          fH_tDpImRXmthS-Ptc_isg   1   1      33026            0     16.9mb          8.7mb        8.7mb
green  open   test.finance.database.wind.asharerdexpenditure            vqJvsf02Sf2j63anKANV3g   1   1     204519            0     93.5mb         46.6mb       46.6mb
green  open   test.finance.database.wind.ashareadvancepayment           Iy2CG2kYS_OK5tWuBqSnNQ   1   1     255946            0    103.4mb         51.6mb       51.6mb
green  open   test.finance.database.wind.ashareloandetails              UTg4VAhoQWCj0wr6CF_DPA   1   1     146251            0     82.3mb         41.5mb       41.5mb
gree

In [9]:
# 看看已入库的法案数量
from elasticsearch.helpers import scan
# 使用 scan API 和 Python 集合去重
all_docs = scan(
    tmpes.es,
    query={"query": {"match_all": {}}},
    index=es_index,
    _source=["celex_number"]  # 只获取法案号字段
)

# 使用集合去重
unique_celex = {doc['_source']['celex_number'] for doc in all_docs if 'celex_number' in doc['_source']}

print("已入库的法案号列表：")
print("-" * 30)
for celex in sorted(unique_celex):  # 排序后打印
    print(celex)
print("-" * 30)
print(f"总共有 {len(unique_celex)} 个不同的法案")

2024-12-12 16:45:53,212 - elastic_transport.transport - INFO - POST https://elastic.kclab.cloud:9200/test.lawagent.regulations_zy/_search?scroll=5m [status:200 duration:0.116s]
2024-12-12 16:45:53,237 - elastic_transport.transport - INFO - POST https://elastic.kclab.cloud:9200/_search/scroll [status:200 duration:0.022s]
2024-12-12 16:45:53,257 - elastic_transport.transport - INFO - DELETE https://elastic.kclab.cloud:9200/_search/scroll [status:200 duration:0.019s]


已入库的法案号列表：
------------------------------
32023R0956
------------------------------
总共有 1 个不同的法案


In [7]:
#看看是否插入成功
# 使用 size 参数确保获取所有文档
query = {
    'query': {'match_all': {}},
    'size': 10000  # 设置一个足够大的数字，或者使用 scan API 处理大量文档
}
res = tmpes.es.search(index=es_index, body=query)
for hit in res['hits']['hits']:
    doc_id = hit['_id']
    celex = hit['_source'].get('celex_number', 'N/A')
    title = hit['_source'].get('law_title', 'N/A')
    content = hit['_source'].get('content', 'N/A')[:50]
    #embedding = hit['_source'].get('embedding', 'N/A')
    print(f"{doc_id:<20} {celex:<20} {title} \n{content}")

2024-12-12 16:43:25,653 - elastic_transport.transport - INFO - POST https://elastic.kclab.cloud:9200/test.lawagent.regulations_zy/_search [status:200 duration:0.120s]


32023R0956_1         32023R0956           Regulation 32023R0956 
THE EUROPEAN PARLIAMENT AND THE COUNCIL OF THE EUR
32023R0956_2         32023R0956           Regulation 32023R0956 
(2)
The Paris Agreement (4), adopted on 12 Decembe
32023R0956_3         32023R0956           Regulation 32023R0956 
(5)
Regulation (EU) 2021/1119 of the European Parl
32023R0956_4         32023R0956           Regulation 32023R0956 
(7)
The Union has been pursuing an ambitious polic
32023R0956_5         32023R0956           Regulation 32023R0956 
(9)
As long as a significant number of the Union’s
32023R0956_6         32023R0956           Regulation 32023R0956 
(11)
Existing mechanisms for addressing the risk o
32023R0956_7         32023R0956           Regulation 32023R0956 
(14)
While the objective of the CBAM is to prevent
32023R0956_8         32023R0956           Regulation 32023R0956 
(18)
With a view to preventing the risk of carbon 
32023R0956_9         32023R0956           Regulation 32023R0956 
(19)
Th

In [8]:
# 然后qa测试
from kernel.qa_core.qa_client import QA
class MyQA(QA):
    def __init__(self):
        super().__init__()
        self.es_client = MyElasticsearchClient()
    
qa_client = MyQA()

questions_list = [
    "本条例不适用于原产于哪些国家的货物？",
    "附录中提到的释放一氧化二氮气体的货物有哪些？",
    "simple goods是什么？",
    "会排放perfluorocarbons（全氟化碳）的货物有哪些？要给出其CN编码 ",
]
gold_recall_docs = [
    'annex III',
    'annex I',
    'annex IV',
    'annex II',
] 
for q in questions_list:
    answer = qa_client.integrated_query_answer(q, [], "32023R0956")
    print(q)
    print(answer)

2024-12-12 16:43:38,530 - langchain_community.utils.user_agent - WARNING - USER_AGENT environment variable not set, consider setting it to identify your requests.
2024-12-12 16:43:45,287 - elastic_transport.transport - INFO - POST https://elastic.kclab.cloud:9200/test.lawagent.regulations_zy/_search [status:200 duration:0.048s]
2024-12-12 16:43:45,314 - elastic_transport.transport - INFO - POST https://elastic.kclab.cloud:9200/test.lawagent.regulations_zy/_search [status:200 duration:0.026s]
2024-12-12 16:43:45,344 - elastic_transport.transport - INFO - POST https://elastic.kclab.cloud:9200/test.lawagent.regulations_zy/_search [status:200 duration:0.029s]
2024-12-12 16:43:45,387 - elastic_transport.transport - INFO - POST https://elastic.kclab.cloud:9200/test.lawagent.regulations_zy/_search [status:200 duration:0.041s]


{'query': ['exemptions', 'countries of origin'], 'filter': None}


2024-12-12 16:43:45,493 - elastic_transport.transport - INFO - POST https://elastic.kclab.cloud:9200/test.lawagent.regulations_zy/_search [status:200 duration:0.034s]
2024-12-12 16:43:45,535 - elastic_transport.transport - INFO - POST https://elastic.kclab.cloud:9200/test.lawagent.regulations_zy/_search [status:200 duration:0.038s]
2024-12-12 16:43:45,572 - elastic_transport.transport - INFO - POST https://elastic.kclab.cloud:9200/test.lawagent.regulations_zy/_search [status:200 duration:0.035s]


本条例不适用于原产于哪些国家的货物？
本条例不适用于原产于以下国家的货物：冰岛、列支敦士登、挪威和瑞士。此外，也不适用于以下地区的货物：比辛根、赫尔戈兰、利维尼奥、休达和梅利利亚（来源: Regulation 32023R0956, ANNEX III）。


2024-12-12 16:43:52,994 - elastic_transport.transport - INFO - POST https://elastic.kclab.cloud:9200/test.lawagent.regulations_zy/_search [status:200 duration:0.025s]
2024-12-12 16:43:53,030 - elastic_transport.transport - INFO - POST https://elastic.kclab.cloud:9200/test.lawagent.regulations_zy/_search [status:200 duration:0.034s]
2024-12-12 16:43:53,061 - elastic_transport.transport - INFO - POST https://elastic.kclab.cloud:9200/test.lawagent.regulations_zy/_search [status:200 duration:0.031s]
2024-12-12 16:43:53,089 - elastic_transport.transport - INFO - POST https://elastic.kclab.cloud:9200/test.lawagent.regulations_zy/_search [status:200 duration:0.026s]
2024-12-12 16:43:53,121 - elastic_transport.transport - INFO - POST https://elastic.kclab.cloud:9200/test.lawagent.regulations_zy/_search [status:200 duration:0.031s]
2024-12-12 16:43:53,153 - elastic_transport.transport - INFO - POST https://elastic.kclab.cloud:9200/test.lawagent.regulations_zy/_search [status:200 duration:0.030s

...

simple goods是什么？
The term "simple goods" refers to goods produced in a production process that requires exclusively input materials (precursors) and fuels having zero embedded emissions (Regulation 32023R0956, ANNEX IV).


2024-12-12 16:44:19,628 - elastic_transport.transport - INFO - POST https://elastic.kclab.cloud:9200/test.lawagent.regulations_zy/_search [status:200 duration:0.039s]
2024-12-12 16:44:19,660 - elastic_transport.transport - INFO - POST https://elastic.kclab.cloud:9200/test.lawagent.regulations_zy/_search [status:200 duration:0.029s]
2024-12-12 16:44:19,691 - elastic_transport.transport - INFO - POST https://elastic.kclab.cloud:9200/test.lawagent.regulations_zy/_search [status:200 duration:0.027s]
2024-12-12 16:44:19,713 - elastic_transport.transport - INFO - POST https://elastic.kclab.cloud:9200/test.lawagent.regulations_zy/_search [status:200 duration:0.019s]


{'query': ['perfluorocarbons', 'CN code'], 'filter': None}


2024-12-12 16:44:19,795 - elastic_transport.transport - INFO - POST https://elastic.kclab.cloud:9200/test.lawagent.regulations_zy/_search [status:200 duration:0.023s]
2024-12-12 16:44:19,815 - elastic_transport.transport - INFO - POST https://elastic.kclab.cloud:9200/test.lawagent.regulations_zy/_search [status:200 duration:0.017s]
2024-12-12 16:44:19,835 - elastic_transport.transport - INFO - POST https://elastic.kclab.cloud:9200/test.lawagent.regulations_zy/_search [status:200 duration:0.017s]


会排放perfluorocarbons（全氟化碳）的货物有哪些？要给出其CN编码 
会排放全氟化碳的货物包括初级铝制品，例如未加工铝、铝粉和薄片、铝棒、型材、铝线、以及厚度超过0.2毫米的铝板、片和带等。它们的对应中国海关商品编码（CN编码）分别是7601、7603、7604、7605和7606（来源：Regulation 32023R0956, ANNEX I）。
